In [24]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import copy
import csv
import re
from typing import List, Optional, Union

"""
This is an EFX allocation finder based on Simulated Annealing.
It uses as objective function the number of EFX violations and the single-transfer
neighborhood structure. The search is initialized with a uniform random allocation.
"""

# Helper functions

def _count_violations_for_pair(envious_idx, envied_idx, v, bundles, agent_utilities):
    """Counts EFX violations for a single ordered pair (i,j) using the utility cache."""
    # Get the envious agent's utility for their own bundle from the cache (O(1) lookup)
    u_envious_own = agent_utilities[envious_idx]

    # Compute the utility for the other bundle on the fly
    envied_bundle = bundles[envied_idx]
    u_envious_of_envied = sum(v[envious_idx][item] for item in envied_bundle)

    violations = 0
    for item_k in envied_bundle:
        if u_envious_of_envied - v[envious_idx][item_k] > u_envious_own:
            violations += 1
    return violations

def calculate_full_potential(n, v, bundles, agent_utilities):
    """Calculates the total potential from scratch. Used only for initialization."""
    total_violations = 0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            total_violations += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
    return total_violations

# Initialization functions
def initialize_allocation(n, m, v):
    """
    Creates a random allocation and all necessary cached data structures.
    Returns:
        bundles (list of sets): bundles[i] is the set of items for agent i.
        owner (list): owner[j] is the agent who owns item j.
        agent_utilities (list): agent_utilities[i] = u_i(A_i).
    """
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    for item_j in range(m):
        agent_i = random.randint(0, n - 1)
        bundles[agent_i].add(item_j)
        owner[item_j] = agent_i
        agent_utilities[agent_i] += v[agent_i][item_j]

    return bundles, owner, agent_utilities

# Potentially better initialization: start with the welfare maximizing allocation
def initialize_allocation_from_optimal_welfare(n, m, v):
    """
    Creates an allocation that maximizes social welfare and all necessary cached data structures.
    Returns:
        bundles (list of sets): bundles[i] is the set of items for agent i.
        owner (list): owner[j] is the agent who owns item j.
        agent_utilities (list): agent_utilities[i] = u_i(A_i).
    """
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    for item_j in range(m):
        agent_i = -1  # will give item_j to the agent that likes it the most
        highest_value_for_j = -1
        for i in range(n):
            if v[i][item_j] > highest_value_for_j:
                agent_i = i
                highest_value_for_j = v[i][item_j]

        bundles[agent_i].add(item_j)
        owner[item_j] = agent_i
        agent_utilities[agent_i] += v[agent_i][item_j]

    return bundles, owner, agent_utilities

def generate_random_valuations_int(n, m):
    """Generates the valuation matrix using random values."""
    return [[random.uniform(1, 1000) for _ in range(m)] for _ in range(n)]

def generate_random_valuations(n, m):
    """Generates the valuation matrix using random values."""
    return [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)]

def sample_correlated_valuations(n, m, correlation_strength=0.8, seed=None):
    """Samples additive valuations for agents that are correlated."""
    if seed is not None:
        np.random.seed(seed)
    assert 0 <= correlation_strength <= 1, "correlation_strength must be between 0 and 1"
    shared = np.random.uniform(0, 1, size=m)
    v = [[0 for _ in range(m)] for _ in range(n)]
    for i in range(n):
        noise = np.random.uniform(0, 1, size=m)
        for j in range(m):
            v[i][j] = correlation_strength * shared[j] + (1 - correlation_strength) * noise[j]
    return v

# --- Functions to save the last instance (valuation and initial/final allocation) to a file in case assert breaks
# Save the valuation matrix to a file
def write_valuation_to_file(n, m, matrix, filename='valuation_matrix.txt'):
    with open(filename, 'w', encoding='utf-8') as f:
        for row in matrix:
            row_str = ' '.join(map(str, row))
            f.write(row_str + '\n')
    return True

# Save the allocation matrix to a file
def write_allocation_to_file(n, m, bundles, filename):
    # Convert the allocation to matrix format
    matrix = [[0 for _ in range(m)] for _ in range(n)]
    for i in range(n):
        for j in bundles[i]:
            matrix[i][j] = 1

    with open(filename, 'w', encoding='utf-8') as f:
        for row in matrix:
            row_str = ' '.join(map(str, row))
            f.write(row_str + '\n')
    return True

# Main efx finder; it receives the instance and an initial allocation to start from (together with some  auxiliary
# data structures like owner list of each item and the precomputed utilities of the agents)
def find_efx_allocation(n, m, v, bundles, owner, agent_utilities):
    # 1. SETUP
    num_steps = 0  # Count the total number of steps until reaching an EFX allocation
    num_restarts = 0

    while True:  # Loop for restarts
        num_restarts += 1

        # 2. INITIALIZATION
        f = calculate_full_potential(n, v, bundles, agent_utilities)

        best_f_value = f
        T = 5.0  # Initial temperature
        T_min = 0.0001
        cooling_rate = 0.99

        # 3. SIMULATED ANNEALING LOOP
        while T > T_min and best_f_value > 0:
            steps_per_temp = 100 * n * m  # number of steps at the current temperature T

            for _ in range(steps_per_temp):
                if best_f_value == 0:
                    break

                # a) Propose a random move
                item_to_move = random.randint(0, m - 1)
                owner_old = owner[item_to_move]
                owner_new = random.randint(0, n - 1)
                while owner_new == owner_old:
                    owner_new = random.randint(0, n - 1)

                # b) Calculate the change in potential (the delta)
                affected_agents = {owner_old, owner_new}

                # Calculate the part of the potential related to these agents BEFORE the move
                old_slice_f = 0
                for i in range(n):
                    for j in affected_agents:
                        if i != j:
                            old_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                for i in affected_agents:
                    for j in range(n):
                        if i != j and j not in affected_agents:
                            old_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)

                # c) Tentatively apply the move to the data structures
                bundles[owner_old].remove(item_to_move)
                bundles[owner_new].add(item_to_move)
                agent_utilities[owner_old] -= v[owner_old][item_to_move]
                agent_utilities[owner_new] += v[owner_new][item_to_move]

                # d) Calculate the part of the potential AFTER the move
                new_slice_f = 0
                for i in range(n):
                    for j in affected_agents:
                        if i != j:
                            new_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                for i in affected_agents:
                    for j in range(n):
                        if i != j and j not in affected_agents:
                            new_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)

                # e) Decide whether to accept the move
                delta_f = new_slice_f - old_slice_f

                accept_move = False
                if delta_f < 0:
                    accept_move = True
                else:
                    if T > 0:
                        acceptance_probability = math.exp(-delta_f / T)
                        if random.uniform(0, 1) < acceptance_probability:
                            accept_move = True

                # f) Finalize or revert the move
                if accept_move:
                    owner[item_to_move] = owner_new
                    f += delta_f
                else:
                    # Revert all changes
                    bundles[owner_new].remove(item_to_move)
                    bundles[owner_old].add(item_to_move)
                    agent_utilities[owner_new] -= v[owner_new][item_to_move]
                    agent_utilities[owner_old] += v[owner_old][item_to_move]

                if f < best_f_value:
                    best_f_value = f

                num_steps += 1

            if best_f_value == 0:

                break  # Found EFX allocation
            T *= cooling_rate

        if best_f_value == 0:
            break

    # Save the final allocation to file for cross-check (optional)
    # write_allocation_to_file(n, m, bundles, 'allocation_final_matrix.txt')

    # Recomputing the objective value of the current allocation to make sure it is zero
    recomputed_f_value = calculate_full_potential(n, v, bundles, agent_utilities)

    assert recomputed_f_value == 0, "The potential should be zero at the end"

    print("EFX allocation found! The allocation is: ", bundles)

    return num_steps, bundles, owner, agent_utilities  # return how many steps it took and the final state

# For comparison also implementing the simple round robin algorithm
def round_robin_allocation(n, m, v):
    """Let the agents choose their favorite items in round-robin fashion (bundles are sets)."""
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    items_remaining = list(range(m))

    while len(items_remaining) > 0:
        # each agent takes their favorite remaining item
        for i in range(n):
            if len(items_remaining) == 0:
                break  # no more items to be allocated
            # sort remaining items in decreasing order of value according to agent i
            items_remaining.sort(key=lambda p: v[i][int(p)], reverse=True)
            j = items_remaining[0]
            bundles[i].add(j)
            owner[j] = i
            agent_utilities[i] += v[i][j]
            items_remaining.remove(j)

    return bundles, owner, agent_utilities

# Check if an allocation is exactly envy-free
def is_envy_free(n, v, bundles, agent_utilities):
    """Checks if the allocation is envy-free."""
    for i in range(n):
        u_i = agent_utilities[i]  # utility of agent i for its own bundle
        for j in range(n):
            if i == j:
                continue
            # utility of agent i for agent j's bundle
            u_ij = sum(v[i][k] for k in bundles[j])
            if u_ij > u_i:  # Agent i envies agent j
                return False
    return True  # the allocation is envy-free

# Some helpers for \Phi
def _weights_from_v(n, m, v):
    """
    Returns (w, W, mu), where:
      w[j] = average value of item j across agents  (Definition 4)
      W = sum_j w[j]
      mu = W / n
    """
    w = [sum(v[i][j] for i in range(n)) / float(n) for j in range(m)]
    W = sum(w)
    mu = W / float(n) if n > 0 else 0.0
    return w, W, mu

def _Y_from_bundles(n, bundles, w):
    """Y_i = sum_{g in A_i} w_g for each agent i."""
    return [sum(w[j] for j in bundles[i]) for i in range(n)]

def _phi_value(Y, mu):
    """Unnormalized Phi(A) = sum_i (Y_i - μ)^2 (Proposition 1)."""
    s = 0.0
    for x in Y:
        d = x - mu
        s += d * d
    return s

def _phi_avg_value(Y, mu, W):
    """Phi_avg(A) = (1/W^2) * sum_i (Y_i - μ)^2 (Definition 4)."""
    if W <= 0.0:
        return 0.0
    return _phi_value(Y, mu) / (W * W)

def _delta_phi_avg_for_move(Y, mu, W, owner_old, owner_new, w_item):
    """
    Delta Phi_avg for transferring a single good g of weight w_item from 'owner_old' to 'owner_new'.

    Using the identity from the proof of Proposition 1 for Phi:
      Delta Phi = 2 w_g (w_g - (Phi_old - Phi_new))
    and then dividing by W^2 to obtain Delta Phi_avg (Definition 4).
    """
    if W <= 0.0:
        return 0.0
    return (2.0 * w_item * (w_item - (Y[owner_old] - Y[owner_new]))) / (W * W)

def _min_weight_in_bundle(bundle, w):
    """Helper: min_{g in bundle} w_g, or +inf if bundle is empty."""
    if not bundle:
        return float('inf')
    return min(w[j] for j in bundle)

# Solver for identical valuations. It does steepest descent on \Phi (Proposition 1 in the paper).

def find_efx_allocation_identical(n, m, v, bundles, owner, agent_utilities):
    """
    Finds an EFX allocation under identical valuations using strict steepest descent on Phi.

    Objective: Phi(A) = sum_i (Y_i - μ)^2, where Y_i = sum_{g in A_i} w_g and μ = (sum_j w_j)/n.
    At each step, we pick the best single-good transfer that strictly decreases Phi. By Proposition 1,
    this process is guaranteed to end at an EFX allocation for identical valuations.

    Parameters are identical to find_efx_allocation; returns the same tuple.
    """
    # Precompute item weights w_j (identical valuations ⇒ take the average, which equals the common value)
    w, W, mu = _weights_from_v(n, m, v)
    Y = _Y_from_bundles(n, bundles, w)

    num_steps = 0

    while True:
        # Choose the worst-off agent (smallest Ψ) – it suffices to check violations against this agent.
        i_min = min(range(n), key=lambda i: Y[i])
        best_move = None
        best_delta_phi = 0.0  # most negative ΔΦ

        # For each potential donor j, check if j violates EFX wrt i_min
        for j in range(n):
            if j == i_min:
                continue

            gap = Y[j] - Y[i_min]  # Ψ_j - Ψ_i
            if gap <= 0.0:
                continue

            # Among items in A_j with w_g < gap, pick the one maximizing improvement magnitude.
            # From ΔΦ = 2 w (w - gap), the (positive) improvement is proportional to w*(gap - w).
            best_item_for_j = None
            best_score = 0.0
            for g in bundles[j]:
                wj = w[g]
                if wj < gap:
                    score = wj * (gap - wj)  # proportional to -ΔΦ/2
                    if score > best_score:
                        best_score = score
                        best_item_for_j = g

            if best_item_for_j is not None:
                w_star = w[best_item_for_j]
                delta_phi = 2.0 * w_star * (w_star - gap)  # < 0
                if delta_phi < best_delta_phi:
                    best_delta_phi = delta_phi
                    best_move = (i_min, j, best_item_for_j)

        # No violating donor found ⇒ allocation is EFX
        if best_move is None:
            break

        # Execute the best strictly-improving move
        i_new, j_old, g_move = best_move
        bundles[j_old].remove(g_move)
        bundles[i_new].add(g_move)
        owner[g_move] = i_new

        # Update utilities (valuations are identical, but we still use v for correctness)
        agent_utilities[j_old] -= v[j_old][g_move]
        agent_utilities[i_new] += v[i_new][g_move]

        # Update Ψ
        w_move = w[g_move]
        Y[j_old] -= w_move
        Y[i_new] += w_move

        num_steps += 1

    # Double-check with the original f(A) (Definition 2)
    potential_recalculated = calculate_full_potential(n, v, bundles, agent_utilities)
    assert potential_recalculated == 0, \
        "Recalculated the potential at the end and it is not zero (identical-valuations solver)."

    print(f"End of Φ-steepest-descent run (identical valuations). Total steps: {num_steps}")
    print("\n>>> EFX allocation found! <<<")

    return num_steps, bundles, owner, agent_utilities

# Interactive setup for n, m, and a valuation matrix

def ask_int(prompt: str, default: int, *, min_value: int = 1, name: Optional[str] = None) -> int:
    """Ask for a positive integer with a default on Enter."""
    label = name or prompt
    while True:
        try:
            raw = input(f"{prompt} [default: {default}] → ").strip()
        except EOFError:
            print(f"\n→ No input stream; using default {label} = {default}")
            return default

        if raw == "":
            print(f"→ Using default {label} = {default}")
            return default

        try:
            val = int(raw)
        except ValueError:
            print(f"Please enter a whole number, or press Enter for {default}.")
            continue

        if val < min_value:
            print(f"Please enter a value ≥ {min_value}, or press Enter for {default}.")
            continue

        print(f"→ Set {label} = {val}")
        return val

Number = Union[int, float]

def ask_matrix(
    n: int,
    m: int,
    *,
    dtype=float,                  # default to float so Enter => [0,1)
    int_range=(1, 10),           # inclusive
    float_range=(0.0, 1.0),      # half-open [lo, hi)
    rand_seed: Optional[int] = None,
    use_generators: bool = True, # call user-defined generators if available (independent only)
    correlation_strength: Optional[float] = None,  # if set, random generation uses this ρ
    default_corr_strength: float = 0.5             # used if user picks 'correlated' and hits Enter
) -> List[List[Number]]:
    """
    Interactively obtain an n×m valuation matrix.

    Options shown to the user:
      • Press Enter → auto-generate random values.
          - dtype=float → uniform in [float_range[0], float_range[1])
          - dtype=int   → randint in [int_range[0], int_range[1]] (inclusive)
        After choosing random, you may select 'independent' or 'correlated' (ρ ∈ [0,1]).
      • Type 'paste' → paste n rows, each with m numbers (comma OR space separated).
      • Provide a CSV filepath → load from file.

    Independent generation:
      - If `use_generators` and `generate_random_valuations` / `_int` exist, they’re used.
      - Otherwise, built-ins are used.

    Correlated generation:
      - ALWAYS calls your `sample_correlated_valuations(n, m, ρ)`.
      - If `dtype=int`, the correlated floats are monotonically mapped to integers in `int_range`.
      - If `sample_correlated_valuations` is not found when requested, a RuntimeError is raised.
    """
    is_int = (dtype == int)
    caster = int if is_int else float

    if rand_seed is not None:
        random.seed(rand_seed)

    # ---- helpers ------------------------------------------------------------

    def _floats_to_ints(M: List[List[float]], lo: int, hi: int) -> List[List[int]]:
        """
        Map floats to integers in [lo, hi] while preserving rank structure.
        We min-max normalize the *actual* matrix to [0,1), then take floor.
        """
        out: List[List[int]] = []
        min_f = min(min(row) for row in M)
        max_f = max(max(row) for row in M)
        denom = (max_f - min_f) if (max_f - min_f) > 0 else 1.0
        span = (hi - lo + 1)
        eps = 1e-12
        for row in M:
            mapped = []
            for x in row:
                z = (x - min_f) / denom            # normalize to [0,1]
                z = min(max(z, 0.0), 1.0 - eps)    # ensure in [0,1)
                mapped.append(lo + int(math.floor(z * span)))
            out.append(mapped)
        return out

    def _random_matrix(rho: Optional[float]) -> List[List[Number]]:
        """
        Generate independent (rho=None) or correlated (rho in [0,1]) matrix.
        Correlated path strictly uses `sample_correlated_valuations`.
        """
        if rho is None:
            # INDEPENDENT
            if not is_int:
                # floats
                if use_generators and "generate_random_valuations" in globals():
                    gen = globals()["generate_random_valuations"]
                    if callable(gen):
                        return gen(n, m)
                lo, hi = map(float, float_range)
                span = hi - lo
                return [[lo + random.random() * span for _ in range(m)] for _ in range(n)]
            else:
                # ints
                if use_generators and "generate_random_valuations_int" in globals():
                    geni = globals()["generate_random_valuations_int"]
                    if callable(geni):
                        return geni(n, m)
                lo_i, hi_i = map(int, int_range)
                return [[random.randint(lo_i, hi_i) for _ in range(m)] for _ in range(n)]
        else:
            # CORRELATED — must use user's function
            if "sample_correlated_valuations" not in globals() or not callable(globals()["sample_correlated_valuations"]):
                raise RuntimeError(
                    "Correlated generation requested but 'sample_correlated_valuations(n, m, correlation_strength)' "
                    "was not found. Please import/define it before calling ask_matrix with correlated mode."
                )
            sampler = globals()["sample_correlated_valuations"]
            M_float = sampler(n, m, float(rho))
            if not is_int:
                return M_float
            # dtype=int: map correlated floats to integer range
            lo_i, hi_i = map(int, int_range)
            return _floats_to_ints(M_float, lo_i, hi_i)

    def _prompt_corr_strength() -> float:
        """Ask user for ρ and validate."""
        while True:
            try:
                s = input(f"Correlation strength ρ in [0,1] (Enter for {default_corr_strength}): ").strip()
            except EOFError:
                return float(default_corr_strength)
            if s == "":
                return float(default_corr_strength)
            try:
                rho_val = float(s)
                if 0.0 <= rho_val <= 1.0:
                    return rho_val
            except ValueError:
                pass
            print("Please enter a number between 0 and 1.")

    # ---- UI -----------------------------------------------------------------
    print(
        f"\nValuation matrix V ({n}×{m}) — choose one of the following options:\n"
        + (
            f"  • Press Enter → auto-generate random *integers* in [{int_range[0]}, {int_range[1]}].\n"
            if is_int else
            f"  • Press Enter → auto-generate random *floats* uniform in [{float_range[0]}, {float_range[1]}).\n"
        )
        + "      After choosing random, you can pick independent or correlated (ρ ∈ [0,1]).\n"
        + f"  • Type 'paste' → you'll enter {n} rows, each with {m} {dtype.__name__}s.\n"
        + f"      Example (n=2, m=3):\n"
        + f"        row 1: 2, 3, 5\n"
        + f"        row 2: 7, 11, 13\n"
        + f"      Tip: commas or spaces are fine: '2 3 5' also works.\n"
        + f"  • Or provide a CSV filepath → I'll load it.\n"
    )

    while True:
        try:
            choice = input("Your choice [Enter | paste | /path/to/file.csv] → ").strip()
        except EOFError:
            print("→ No input stream; generating a random matrix.")
            rho = None if correlation_strength is None else float(correlation_strength)
            V = _random_matrix(rho)
            label = "correlated" if rho is not None else "independent"
            if rho is not None:
                print(f"→ Generated random {label} V (ρ={rho}) of shape {n}×{m}.")
            else:
                print(f"→ Generated random {label} V of shape {n}×{m}.")
            return V

        # 1) Random (independent or correlated)
        if choice == "":
            rho = None if correlation_strength is None else float(correlation_strength)
            if rho is None:
                try:
                    mode = input("Random mode → independent or correlated? [i/c] (Enter for i) → ").strip().lower()
                except EOFError:
                    mode = ""
                if mode in ("c", "corr", "correlated"):
                    rho = _prompt_corr_strength()
            V = _random_matrix(rho)
            label = "correlated" if rho is not None else "independent"
            if rho is not None:
                print(f"→ Generated random {label} V (ρ={rho}) of shape {n}×{m}.")
            else:
                print(f"→ Generated random {label} V of shape {n}×{m}.")
            return V

        # 2) Paste (guided)
        if choice.lower() == "paste":
            print(f"Paste mode: enter {n} rows of {m} values (commas or spaces). Use the example above.")
            rows: List[List[Number]] = []
            while len(rows) < n:
                line = input(f"row {len(rows)+1}: ").strip()
                if not line:
                    print("Empty line—please enter the numbers as shown in the example.")
                    continue
                parts = [p for p in re.split(r"[,\s]+", line) if p]
                try:
                    vals = [caster(p) for p in parts]
                except ValueError:
                    print("Couldn't parse that row—use only numbers separated by commas/spaces.")
                    continue
                if len(vals) != m:
                    print(f"Expected {m} values, got {len(vals)}. Try again.")
                    continue
                rows.append(vals)
            print("→ Matrix captured from paste.")
            return rows

        # 3) CSV file
        path = choice.strip().strip("'\"")  # tolerate quoted paths
        try:
            with open(path, newline="") as f:
                sample = f.read(4096)
                f.seek(0)
                try:
                    dialect = csv.Sniffer().sniff(sample)
                except csv.Error:
                    dialect = csv.excel
                reader = csv.reader(f, dialect)
                V: List[List[Number]] = []
                for row in reader:
                    if not row:
                        continue
                    values: List[Number] = []
                    for x in row:
                        x = x.strip()
                        if x == "":
                            continue
                        values.append(caster(x))
                    if values:
                        V.append(values)
        except FileNotFoundError:
            print("File not found. Try again, or press Enter for random.")
            continue
        except PermissionError:
            print("Permission denied opening that file. Try a different path, or press Enter for random.")
            continue
        except ValueError:
            print("Couldn't parse numbers in that file. Try again, or press Enter for random.")
            continue

        # Validate shape
        if len(V) != n or any(len(r) != m for r in V):
            got_rows = len(V)
            got_cols = len(V[0]) if V else 0
            print(f"CSV shape was {got_rows}×{got_cols}, expected {n}×{m}. Try again.")
            continue

        print(f"→ Loaded matrix from {path}.")
        return V

if __name__ == '__main__':

   # Initialization
   print("# Initialization")

   n = ask_int("Number of agents (n)", default=5, min_value=1, name="n")

   m = ask_int("Number of items (m)",  default=30, min_value=1, name="m")

   v = ask_matrix(n, m)

   print("n =" + str(n) + "\n")

   print("m =" + str(m) + "\n")

   #v = generate_random_valuations(n, m) # If correlated valuations are desired, replace this line with: v = sample_correlated_valuations(n, m, correlation_strength)

   # ------- Running SA on the same valuations but initializing randomly the allocation
   bundles_opt, owner_opt, agent_utilities_opt = initialize_allocation(n, m, v)
   print("Valuation matrix:" + str(v))
   print("\n")
   print("Initial allocation:" + str(bundles_opt))
   print("\n \n")

   start_time_opt = time.time()

   num_steps_opt, bundles_end_opt, owner_end_opt, agent_utilities_end_opt = find_efx_allocation(n, m, v, bundles_opt, owner_opt, agent_utilities_opt)

   end_time_opt = time.time()

   total_time_opt = end_time_opt - start_time_opt

   print("Runtime " + str(total_time_opt) + " seconds. The number of steps taken is " + str(num_steps_opt) + "\n")



# Initialization
Number of agents (n) [default: 5] → 4
→ Set n = 4
Number of items (m) [default: 30] → 19
→ Set m = 19

Valuation matrix V (4×19) — choose one of the following options:
  • Press Enter → auto-generate random *floats* uniform in [0.0, 1.0).
      After choosing random, you can pick independent or correlated (ρ ∈ [0,1]).
  • Type 'paste' → you'll enter 4 rows, each with 19 floats.
      Example (n=2, m=3):
        row 1: 2, 3, 5
        row 2: 7, 11, 13
      Tip: commas or spaces are fine: '2 3 5' also works.
  • Or provide a CSV filepath → I'll load it.

Your choice [Enter | paste | /path/to/file.csv] → 
Random mode → independent or correlated? [i/c] (Enter for i) → c
Correlation strength ρ in [0,1] (Enter for 0.5): 0.3
→ Generated random correlated V (ρ=0.3) of shape 4×19.
n =4

m =19

Valuation matrix:[[np.float64(0.5410666366419821), np.float64(0.14683553867646768), np.float64(0.1215579257272975), np.float64(0.41744819714272174), np.float64(0.7816498840684649), np.fl